In [1]:
#Day 08 p. 149 16:00~16:15(저장)

#numpy csv 파일 저장

import numpy as np

np.random.seed(42)

a = np.random.randn(3,4)
a[2][2] = np.nan
print(a)

[[ 0.49671415 -0.1382643   0.64768854  1.52302986]
 [-0.23415337 -0.23413696  1.57921282  0.76743473]
 [-0.46947439  0.54256004         nan -0.46572975]]


In [2]:
np.savetxt('np.csv', a, fmt='%.2f', delimiter = ',' ,header = "#1, #2, #3, #4")

In [10]:
#pandas  csv 파일 저장
import pandas as pd

df = pd.DataFrame(a)
print(df)

df.to_csv('pd.csv', float_format='%.2f', na_rep="NAN!")

          0         1         2         3
0  0.496714 -0.138264  0.647689  1.523030
1 -0.234153 -0.234137  1.579213  0.767435
2 -0.469474  0.542560       NaN -0.465730


In [14]:
#.npy와 피클 포맷 ~180 / 16:23 ~18:55
import numpy as np
import pandas as pd
from tempfile import NamedTemporaryFile
from os.path import getsize

np.random.seed(42)
a = np.random.randn(365,4)

#csv 파일에 데이터를 저장 후 크기를 확인
tmpf = NamedTemporaryFile()
np.savetxt(tmpf, a, delimiter =',')
print("Size CSV file", getsize(tmpf.name))

#.npy 포맷으로 저장해 배열 불러옴 / csv 크기의 1/3 정도임.
tmpf = NamedTemporaryFile()
np.save(tmpf, a)
tmpf.seek(0)
loaded = np.load(tmpf)
print("Shape", loaded.shape)
print("Size .npy file", getsize(tmpf.name))

#pickle 데이터 만들기. 사이즈 확인 .npy 큰 것을 알수 있다.
df = pd.DataFrame(a)
df.to_pickle(tmpf.name)
print("Size pickled dataframe", getsize(tmpf.name))
print("DF from pickle \n", pd.read_pickle(tmpf.name))


Size CSV file 36693
Shape (365, 4)
Size .npy file 11808
Size pickled dataframe 12276
DF from pickle 
             0         1         2         3
0    0.496714 -0.138264  0.647689  1.523030
1   -0.234153 -0.234137  1.579213  0.767435
2   -0.469474  0.542560 -0.463418 -0.465730
3    0.241962 -1.913280 -1.724918 -0.562288
4   -1.012831  0.314247 -0.908024 -1.412304
5    1.465649 -0.225776  0.067528 -1.424748
6   -0.544383  0.110923 -1.150994  0.375698
7   -0.600639 -0.291694 -0.601707  1.852278
8   -0.013497 -1.057711  0.822545 -1.220844
9    0.208864 -1.959670 -1.328186  0.196861
10   0.738467  0.171368 -0.115648 -0.301104
11  -1.478522 -0.719844 -0.460639  1.057122
12   0.343618 -1.763040  0.324084 -0.385082
13  -0.676922  0.611676  1.031000  0.931280
14  -0.839218 -0.309212  0.331263  0.975545
15  -0.479174 -0.185659 -1.106335 -1.196207
16   0.812526  1.356240 -0.072010  1.003533
17   0.361636 -0.645120  0.361396  1.538037
18  -0.035826  1.564644 -2.619745  0.821903
19   0.087047 -0.2

In [18]:
#PyTables 데이터 저장
#HDF(계층형 데이터 포맷)은 숫자로 이뤄진 빅데이터를 저장하기 위한 기술이다. 
#구조는 데이터 그룹과 데이터셋(다차원 동질 배열homogeneous) 형태 이루어짐
#설치: brew install hdf5 
#pip install numexpr tables

import numpy as np
import tables
from tempfile import NamedTemporaryFile
from os.path import getsize

np.random.seed(42)
a = np.random.randn(365, 4)

tmpf = NamedTemporaryFile()
h5file = tables.open_file(tmpf.name, mode='w', title="NumPy Array")
root = h5file.root
h5file.create_array(root, "array", a)
h5file.close()

#HDF5 파일을 읽고 크기 출력
h5file = tables.open_file(tmpf.name, "r")
print(getsize(tmpf.name))

for node in h5file.root:
   b = node.read()
   print(type(b), b.shape)

h5file.close()


13824
<class 'numpy.ndarray'> (365, 4)


In [25]:
#HDF5 형태로 Pandas 데이터프레임 읽고 쓰기

#데모 파일에 HDFStore 생성자를 가져와서 store 변수에 담는다.
filename = "pytable_df_demo.h5"
store = pd.io.pytables.HDFStore(filename)
print(store)

<class 'pandas.io.pytables.HDFStore'>
File path: pytable_df_demo.h5



In [26]:
store['df'] = df
print(store)

<class 'pandas.io.pytables.HDFStore'>
File path: pytable_df_demo.h5



In [27]:
#데이터프레임에서 GET, 딕셔너리 검색, 단순 점(dot) 접근가능 => 출력 값은 동일
print("Get", store.get('df').shape)
print("Lookup", store['df'].shape)
print("Dotted", store.df.shape)

Get (365, 4)
Lookup (365, 4)
Dotted (365, 4)


In [28]:
del store['df']
print("After del \n", store)

After del 
 <class 'pandas.io.pytables.HDFStore'>
File path: pytable_df_demo.h5



In [29]:
# 저장소가 열려 있는지 알려준다. 닫히면 'False'
print("Before close", store.is_open)
store.close()
print("After close", store.is_open)

Before close True
After close False


In [31]:
# 고정된 형식(속도는 빠르나 추가 및 검색은 불가능) / table은 가능
df.to_hdf(tmpf.name, 'data', format='table')
print(pd.read_hdf(tmpf.name, 'data', where = ['index>363']))

            0         1         2         3
364  0.753342  0.381158  1.289753  0.673181


In [33]:
import numpy as np
import pandas as pd

np.random.seed(42)
a = np.random.randn(365,4)

filename = "pytable_df_demo.h5"
store = pd.io.pytables.HDFStore(filename)
print(store)

df = pd.DataFrame(a)
store['df'] = df
print(store)

print("Get", store.get('df').shape)
print("Lookup", store['df'].shape)
print("Dotted", store.df.shape)

del store['df']
print("After del \n", store)

print("Before close", store.is_open)
store.close()
print("After close", store.is_open)

df.to_hdf('test.h5', 'data', format='table')
print(pd.read_hdf('test.h5', 'data', where = ['index>363']))

<class 'pandas.io.pytables.HDFStore'>
File path: pytable_df_demo.h5

<class 'pandas.io.pytables.HDFStore'>
File path: pytable_df_demo.h5

Get (365, 4)
Lookup (365, 4)
Dotted (365, 4)
After del 
 <class 'pandas.io.pytables.HDFStore'>
File path: pytable_df_demo.h5

Before close True
After close False
            0         1         2         3
364  0.753342  0.381158  1.289753  0.673181


In [34]:
#pandas로 엑셀 파일 읽기 및 쓰기
#pip install openpyxl xlsxwriter xlrd

import numpy as np
import pandas as pd

np.random.seed(42)
a = np. random.randn(365, 4)

filename="excel_demo.xlsx"
df = pd.DataFrame(a)
print(filename)
df.to_excel(filename, sheet_name='Random Data')
print("Means\n", pd.read_excel(filename, 'Random Data').mean())

excel_demo.xlsx
Means
 Unnamed: 0    182.000000
0               0.037860
1               0.024483
2               0.059836
3               0.058417
dtype: float64


In [45]:
#REST 웹서비스 및 JSON 저장
#웹 표현 상태변경(Rest) Javescript Object Notation

import json

json_str = '{"country":"Netherlands","dma_code":"0","timezone":"Europe\/Amsterdam","area_code":"0","ip":"46.19.37.108","asn":"AS196752","continent_code":"EU","isp":"Tilaa V.O.F.","longitude":5.75,"latitude":52.5,"country_code":"NL","country_code3":"NLD"}'

#json 사용함수 loads
data = json.loads(json_str)

#country 접근
print("Country", data["country"])

#값을 덮어씀
data["country"] = "Brazil"
print(json.dumps(data))

Country Netherlands
{"asn": "AS196752", "country": "Brazil", "continent_code": "EU", "isp": "Tilaa V.O.F.", "latitude": 52.5, "area_code": "0", "timezone": "Europe/Amsterdam", "longitude": 5.75, "country_code3": "NLD", "country_code": "NL", "dma_code": "0", "ip": "46.19.37.108"}


In [47]:
#pandas에서 json 읽기 및 쓰기

import pandas as pd

json_str = '{"country":"Netherlands","dma_code":"0","timezone":"Europe\/Amsterdam","area_code":"0","ip":"46.19.37.108","asn":"AS196752","continent_code":"EU","isp":"Tilaa V.O.F.","longitude":5.75,"latitude":52.5,"country_code":"NL","country_code3":"NLD"}'

data = pd.read_json(json_str, typ='series')
print("Series \n", data)

data["country"] = "Brazil"
print("New Series \n", data.to_json())

Series 
 area_code                        0
asn                       AS196752
continent_code                  EU
country                Netherlands
country_code                    NL
country_code3                  NLD
dma_code                         0
ip                    46.19.37.108
isp                   Tilaa V.O.F.
latitude                      52.5
longitude                     5.75
timezone          Europe/Amsterdam
dtype: object
New Series 
 {"area_code":"0","asn":"AS196752","continent_code":"EU","country":"Brazil","country_code":"NL","country_code3":"NLD","dma_code":"0","ip":"46.19.37.108","isp":"Tilaa V.O.F.","latitude":52.5,"longitude":5.75,"timezone":"Europe\/Amsterdam"}


In [51]:
#RSS Atom 피드 파싱
#pip install feedparser

import feedparser as fp
rss = fp.parse("http://www.packtpub.com/rss.xml")
print("# Entries", len(rss.entries))

# Entries 10


In [56]:
for i, entry in enumerate(rss.entries):
    if "Python" in entry.summary:
        print(i, entry.title)
        print(entry.summary)
    

In [54]:
for i, entry in enumerate(rss.entries):
   if "Java" in entry.summary:
      print(i, entry.title)
      print(entry.summary)

6 FastTrack to OOP - Classes and Interfaces
<p>In this article by <strong>Mohamed Sanaulla</strong> and <strong>Nick Samoylov</strong>, the authors of <a href="https://www.packtpub.com/application-development/java-9-cookbook">Java 9 Cookbook</a>, we will cover the following recipe:</p>
<ul>
<li>Implementing object-oriented design using classes</li>
</ul>
<p style="margin-left: 40px; margin-right: 40px;" align="center"><em>(For more resources related to this topic, see <a href="https://www.packtpub.com/application-development/java-9-cookbook">here</a>.)</em></p>
<p><span style="background-color: transparent;"><strong>Implementing object-oriented design using classes</strong> </span></p>
<p><span style="background-color: transparent;">In this recipe, you will learn about the first two OOD concepts--object/class and encapsulation. </span></p>
<p><span style="background-color: transparent;"><strong>Getting ready</strong> </span></p>
<p><a href="http://www.packtpub.com/books/content/fasttra

In [60]:
#Beautiful Soup 활용한 HTML 파싱
#pip install beautifulsoup4 lxml

from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(open('ex/loremIpsum.html'))
print("First div\n", soup.div)

First div
 <div class="tile">
<h4>Development</h4>
     0.10.1 - July 2014<br/>
</div>


In [62]:
print("First div class", soup.div['class'])

First div class ['tile']


In [63]:
print("First dfn text", soup.dl.dt.dfn.text)

First dfn text Quare attende, quaeso.


In [65]:
for link in soup.find_all('a'):
    print("Link text", link.string, "URL", link.get('href'))

Link text loripsum.net URL http://loripsum.net/
Link text Poterat autem inpune; URL http://loripsum.net/
Link text Is es profecto tu. URL http://loripsum.net/


In [67]:
for i, div in enumerate(soup('div')):
    print(i, div.contents)

0 ['\n', <h4>Development</h4>, '\n     0.10.1 - July 2014', <br/>, '\n']
1 ['\n', <h4>Official Release</h4>, '\n     0.10.0 June 2014', <br/>, '\n']
2 ['\n', <h4>Previous Release</h4>, '\n     0.09.1 June 2013', <br/>, '\n']


In [68]:
official_div = soup.find_all("div", id ="official")
print("official Version", official_div[0].contents[2].strip())

official Version 0.10.0 June 2014


In [69]:
print("# elements with class", len(soup.find_all(class_ =True)))

# elements with class 3


In [76]:
#tile 클래스 갯수 세기
title_class = soup.find_all("div", class_="tile")
print("# Tile classes", len(title_class))

# Tile classes 2


In [78]:
# div 정규 표현식을 사용해 검색하기
print("# Divs with class containing tile", len(soup.find_all("div", class_= re.compile("tile"))))

# Divs with class containing tile 3


In [79]:
#CSS 패턴 정의
print("using CSS selector \n", soup.select('div.notile'))

using CSS selector 
 [<div class="notile">
<h4>Previous Release</h4>
     0.09.1 June 2013<br/>
</div>]


In [80]:
#정렬 리스트에서 첫번째 두개의 아이템 선택
print("Selecting ordered list list items\n", soup.select("ol > li")[:2])

Selecting ordered list list items
 [<li>Cur id non ita fit?</li>, <li>In qua si nihil est praeter rationem, sit in una virtute finis bonorum;</li>]


In [85]:
print("Second list item in ordered list", soup.select("ol > li:nth-of-type(2)"))

Second list item in ordered list [<li>In qua si nihil est praeter rationem, sit in una virtute finis bonorum;</li>]


In [86]:
print("Searching for txt string", soup.find_all(text =re.compile("2014")))

Searching for txt string ['\n     0.10.1 - July 2014', '\n     0.10.0 June 2014']
